In [445]:
import pandas as pd
import numpy as np
import json
import zipfile
import os
import kaggle

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [446]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
import seaborn as sns

In [499]:
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
!kaggle datasets list -s 'Heritage'
!kaggle datasets download -d 'joebeachcapital/unesco-world-heritage-sites'
!unzip joebeachcapital/unesco-world-heritage-sites.zip
df1 = pd.read_csv('joebeachcapital/unesco-world-heritage-sites.csv')

In [ ]:
!kaggle datasets download -d 'joebeachcapital/unesco-world-heritage-sites' --force

In [ ]:
!unzip /Users/katyakraft/unesco-world-heritage-sites.zip -d /Users/katyakraft/unesco-world-heritage-sites

In [451]:
df = pd.read_csv('/Users/katyakraft/unesco-world-heritage-sites/whc-sites-2023.csv')

In [ ]:
df.drop(columns= ["rev_bis", "unique_number", "id_no", "name_fr", "short_description_en", "short_description_fr", "justification_en", "justification_fr", "criteria_txt", "category_short", "states_name_fr", "region_en", "region_fr", "udnp_code", "transboundary", "secondary_dates", "danger", "date_end", "danger_list", "longitude", "latitude", "area_hectares","C1", "C2", "C3", "C4", "C5", "C6", "N7", "N8", "N9", "N10"], inplace = True)
df = df.rename(columns={"name_en": "name", "states_name_en": "country"})
df = df[["iso_code", "country", "name", "date_inscribed", "category"]]

In [453]:
df.groupby("country")
df['iso_code'] = df['iso_code'].str.split(',')
df['country'] = df['country'].str.split(',')
df = df.explode(['iso_code', 'country']).reset_index(drop=True)
df['iso_code'] = df['iso_code'].str.upper()

In [455]:
df['country'] = df['country'].replace({
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Congo': 'Congo, Dem. Rep.',
    'Czechia': 'Czech Republic',
    'Egypt': 'Egypt, Arab Rep.',
    'Gambia': 'Gambia, The',
    'Iran (Islamic Republic of)': 'Iran, Islamic Rep.',
    'Republic of Korea': 'Korea, Rep.',
    'Kyrgyzstan': 'Kyrgyz Republic',
    "Lao People's Democratic Republic": 'Lao PDR',
    'Micronesia (Federated States of)': 'Micronesia, Fed. Sts.',
    'Republic of Moldova': 'Moldova',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'United Republic of Tanzania': 'Tanzania',
    'United States of America': 'United States',
    'Venezuela (Bolivarian Republic of)': 'Venezuela, RB',
    'Viet Nam': 'Vietnam',
    'Yemen': 'Yemen, Rep.',
    'Türkiye': 'Turkey'
})

In [ ]:
df_country_counts = df.groupby('country').agg(
    site_count=('name', 'size'),                   
    date_inscribed=('date_inscribed', lambda x: ', '.join(map(str, sorted(x)))),  # All inscription dates as a string
).reset_index()  
df_country_counts = df_country_counts.sort_values(by='country', ascending=True).reset_index(drop=True)
df_country_counts.head()

In [469]:
df_arrivals = pd.read_csv('/Users/katyakraft/Documents/IronHack/Final_project/API_ST.INT.ARVL_DS2_en_csv_v2_1927083.csv')

In [ ]:
df_arrivals = df_arrivals.loc[:, ~df_arrivals.columns.isin(map(str, range(1960, 2007)))]
df_arrivals = df_arrivals.drop(columns=["2019", "2020"])
new_column_names = {f"{year}": f"arrivals_{year}" for year in range(2007, 2019)}
new_column_names["Country Name"] = "country"
df_arrivals = df_arrivals.rename(columns=new_column_names)
df_arrivals.columns = df_arrivals.columns.str.replace(' ', '_').str.lower()
arrival_columns = [f'arrivals_{year}' for year in range(2007, 2019)]
df_arrivals['total_arrivals'] = df_arrivals[arrival_columns].sum(axis=1)
df_arrivals = df_arrivals.fillna(0)

In [471]:
# Drop not relevant data like regions. 
df_arrivals.drop([5, 32, 34, 47, 59, 60, 61, 62, 63, 66, 71, 72, 93, 96, 100, 101, 102, 103, 105, 108, 126, 132, 133, 134, 137, 138, 140, 145, 151, 154, 159, 168, 179, 181, 189, 195, 196, 202, 213, 215, 216, 228, 229 ,234, 236, 238, 239, 247, 257], inplace=True)
pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
df_combined = pd.merge(df_country_counts, df_arrivals, on='country', how='outer')
df_combined = df_combined.fillna("0")

NEW Dataframe with all date_inscribed per country

In [ ]:
df_country_counts_2 = pd.DataFrame(df_country_counts)

# Years to create columns for
years = range(2007, 2019)

# Create columns for each year, initialized to 0 or False
for year in years:
    df_country_counts_2[f'inscribed_{year}'] = df_country_counts_2['date_inscribed'].apply(
        lambda x: 1 if str(year) in x.split(', ') else 0 )
df_country_counts_2 = df_country_counts_2.drop(columns = "date_inscribed")

In [503]:
df_combined_2 = pd.merge(df_country_counts_2, df_arrivals, on='country', how='right')
df_combined_2= df_combined_2.fillna("0")
df_filtered = df_combined_2[df_combined_2['total_arrivals'] != 0]

In [479]:
df_filtered.to_csv("/Users/katyakraft/Documents/IronHack/Final_project/data_frames/merged_tourism_data_2.csv", index=False)

In [ ]:
df_filtered.head()

# HYPOTHESIS TESTING 

# 1. Hypothesis

The Relationship Between UNESCO World Heritage Sites and Tourism Arrivals.

(H₀): There is no significant correlation between the number of UNESCO sites and the number of tourist arrivals.

(H₁): There is a significant positive correlation between the number of UNESCO sites and the number of tourist arrivals.

In [ ]:
df_filtered['site_count'] = pd.to_numeric(df_filtered['site_count'], errors='coerce')
df_filtered['total_arrivals'] = pd.to_numeric(df_filtered['total_arrivals'], errors='coerce')
df_filtered = df_filtered.dropna(subset=['site_count', 'total_arrivals'])

# Drop rows with missing values
df_filtered = df_filtered.dropna(subset=['site_count', 'total_arrivals'])

# Plot
plt.figure(figsize=(10, 6))
sns.regplot(x='site_count', y='total_arrivals', data=df_filtered, line_kws={"color": "red"})
plt.title('UNESCO Site Count vs. Total Tourism Arrivals')
plt.xlabel('Number of UNESCO Sites')
plt.ylabel('Total Tourism Arrivals')
plt.show()

# H1: Pearsons Correlation Test

In [ ]:
# Convert relevant columns to numeric
arrival_columns = [f'arrivals_{year}' for year in range(2007, 2019)]
inscription_columns = [f'inscribed_{year}' for year in range(2007, 2019)]

for column in arrival_columns + inscription_columns:
    df_filtered[column] = pd.to_numeric(df_filtered[column], errors='coerce')


#Calculate Pearson correlation
correlation, p_value = pearsonr(df_filtered['site_count'], df_filtered['total_arrivals'])

print(f"Pearson Correlation Coefficient: {correlation:.4f}") 
print(f"P-value: {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant positive correlation between the number of UNESCO sites and tourism arrivals.")
else:
    print("Fail to reject the null hypothesis: No significant correlation between the number of UNESCO sites and tourism arrivals.")


#Outcome: 
#Pearson Correlation Coefficient: 0.8089
#P-value: 0.0000
#Reject the null hypothesis: There is a significant positive correlation between the number of UNESCO sites and tourism arrivals.


# 2. Hypothesis 

Low Tourism and Fewer UNESCO Sites

Hypothesis Statements

(H₀): There is no association between lower tourism arrivals and a lower count of UNESCO World Heritage sites.

(H₁): Countries with lower tourism arrivals tend to have fewer UNESCO World Heritage sites.

In [ ]:
# Creating a new column categorizing low and high UNESCO site counts
df_filtered['site_category'] = df_filtered['site_count'].apply(lambda x: 'Low' if x < 5 else 'High')

plt.figure(figsize=(10, 6))
sns.boxplot(x='site_category', y='total_arrivals', data=df_filtered)
plt.title('Tourism Arrivals by Low and High UNESCO Site Counts')
plt.xlabel('UNESCO Site Category')
plt.ylabel('Total Tourism Arrivals')
plt.show()

# H2: Spearman's Correlation Test

In [ ]:
spearman_corr, p_value = spearmanr(df_filtered['total_arrivals'], df_filtered['site_count'])

print(f"Spearman Correlation Coefficient: {spearman_corr:.4f}")
print(f"P-value: {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant association between lower tourism arrivals and fewer UNESCO World Heritage sites.")
else:
    print("Fail to reject the null hypothesis: No significant association between lower tourism arrivals and fewer UNESCO World Heritage sites.")

#Outcome: 
#Spearman Correlation Coefficient: 0.7030
#P-value: 0.0000
#Reject the null hypothesis: There is a significant association between lower tourism arrivals and fewer UNESCO World Heritage sites.

# 3. Hypothesis 

Comparing the difference in tourism arrivals between countries that have 10 or more UNESCO World Heritage sites and those with fewer.

(H₀): There is no significant difference in tourism arrivals between countries with 10 or more UNESCO sites and those with fewer than 10 sites.

(H₁): Alternative Hypothesis (H1): Countries with 10 or more UNESCO sites attract significantly higher tourism arrivals than those with fewer than 10 sites.

In [ ]:
# Creating a new column categorizing 10 or more sites
df_filtered['ten_or_more_sites'] = df_filtered['site_count'].apply(lambda x: '10 or more' if x >= 10 else 'Less than 10')

# Violin plot
plt.figure(figsize=(10, 6))
sns.violinplot(x='ten_or_more_sites', y='total_arrivals', data=df_filtered)
plt.title('Tourism Arrivals for Countries with 10 or More vs. Fewer than 10 UNESCO Sites')
plt.xlabel('UNESCO Site Count Category')
plt.ylabel('Total Tourism Arrivals')
plt.show()

# H3: Two-sample T-test

In [ ]:
#Split data into two groups based on `site_count`
group_high = df_filtered[df_filtered['site_count'] >= 10]['total_arrivals']
group_low = df_filtered[df_filtered['site_count'] < 10]['total_arrivals']

t_stat, p_value = ttest_ind(group_high, group_low, equal_var=False) 

print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: Countries with 10 or more UNESCO sites attract significantly higher tourism arrivals.")
else:
    print("Fail to reject the null hypothesis: No significant difference in tourism arrivals between countries with 10 or more UNESCO sites and those with fewer.")

#Outcome: 
#T-statistic: 5.2577
#P-value: 0.0000
#Reject the null hypothesis: Countries with 10 or more UNESCO sites attract significantly higher tourism arrivals.

# 4. Hypothesis 

New Site Impact on Tourism.

(H₀): There is no significant increase in tourism arrivals in the year following the announcement of a new UNESCO site compared to the previous year.

(H₁): Tourism arrivals significantly increase in the year following the announcement of a new UNESCO site compared to the previous year.

In [518]:
# Melting the DataFrame to focus on inscription years
inscription_years = df_filtered.melt(id_vars=['country', 'site_count'], 
                                        value_vars=inscription_columns,
                                        var_name='year', 
                                        value_name='inscription')

# Removing rows where the inscription is 0 or NaN
inscription_years = inscription_years[inscription_years['inscription'] != 0]

In [520]:
# Create lists for the arrivals data
arrivals_before_list = []
arrivals_after_list = []
country_list = []  
year_list = []  

#Loop through each row in the melted DataFrame
for index, row in inscription_years.iterrows():
    country = row['country']
    year_inscribed = int(row['year'].split('_')[1])  # Extract the year from the column name
    
    if year_inscribed - 1 >= 2007 and year_inscribed + 1 <= 2018:
        # Get arrivals for the year before and after the inscription year
        arrivals_before = df_filtered.loc[df_filtered['country'] == country, f'arrivals_{year_inscribed - 1}'].values
        arrivals_after = df_filtered.loc[df_filtered['country'] == country, f'arrivals_{year_inscribed + 1}'].values
        
        if arrivals_before.size > 0 and arrivals_after.size > 0:
            # Append the values to the lists
            arrivals_before_list.append(arrivals_before[0])
            arrivals_after_list.append(arrivals_after[0])
            country_list.append(country)  
            year_list.append(year_inscribed)  

# Comparison DataFrame from the lists
comparison_df = pd.DataFrame({
    'country': country_list,
    'year': year_list,
    'arrivals_before': arrivals_before_list,
    'arrivals_after': arrivals_after_list
})

# Removing rows where arrivals_before or arrivals_after are zero
comparison_df = comparison_df[(comparison_df['arrivals_before'] != 0) & (comparison_df['arrivals_after'] != 0)]

#Claculating the percentage change in arrivals
comparison_df['percent_change'] = (
    (comparison_df['arrivals_after'] - comparison_df['arrivals_before']) / comparison_df['arrivals_before']
) * 100

In [ ]:
comparison_df_sorted = comparison_df.sort_values('percent_change', ascending=False)

plt.figure(figsize=(12, 6))
plt.bar(comparison_df_sorted['country'], comparison_df_sorted['percent_change'], color='green')
plt.xlabel('Country')
plt.ylabel('Percentage Change in Arrivals')
plt.title('Percentage Change in Tourism Arrivals After New UNESCO Site Inscription')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# H4: Paired T-test

In [ ]:
t_stat, p_value = ttest_rel(comparison_df['arrivals_before'], comparison_df['arrivals_after'])

print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}") 

alpha = 0.05 

if p_value < alpha:
    print("Reject the null hypothesis: Tourism arrivals significantly increase in the year following the announcement of a new UNESCO site.")
else:
    print("Fail to reject the null hypothesis: No significant increase in tourism arrivals following the announcement of a new UNESCO site.")

#Outcome: 
#T-statistic: -7.1748
#P-value: 0.0000
#Reject the null hypothesis: Tourism arrivals significantly increase in the year following the announcement of a new UNESCO site.

# Merged Data of Hypothesis 4

For final analysis of hypothesis 4 in Tableau I merged the data frame and kept relevant columns before exporting. 

In [ ]:
merged_df = comparison_df.merge(df_country_counts_2[['country', 'site_count']], on='country', how='left')

# Create 'site_category' based on the number of UNESCO sites
merged_df['site_category'] = merged_df['site_count'].apply(lambda x: '10 < ' if x >= 10 else '0-10')

In [498]:
merged_df.to_csv('/Users/katyakraft/Documents/IronHack/Final_project/data_frames/comparison_data_hyp_4_final.csv', index=False)